In [1]:
### Pour voir les métiers recommandés à un élève, on utilise la fonction show_score(eleve, n_metiers)
### --> eleve : liste des critères choisis par l'élève
### --> n_metiers : nombre de métiers à montrer

import io
import os
import json
import csv
import pandas as pd
import numpy as np

In [2]:
# Préparation des chemins utiles
# Main directory
dir_path = os.getcwd()
# Dossier des données
data_dir_path = os.path.join(dir_path, 'Donnees')
# Métiers de chaque cluster :
jobs_clusters = os.path.join(data_dir_path, 'jobs_clusters.csv')
# Tableau des clusters-variables :
df_mean_file = os.path.join(data_dir_path, 'df_mean.csv')
# Tableau des métiers-variables normalisé:
df_clusters_file = os.path.join(data_dir_path, 'df_clusters.csv')

# Fichier excel de l'arborescences des activités
act_csv = os.path.join(data_dir_path,'rome_arboactivités_juin62181.xlsx')
# Fichier excel de l'arborescence des compétences
comp_csv = os.path.join(data_dir_path,'rome_arbocompétences_juin62182.xlsx')

# Données des métiers
df_clusters = pd.read_csv(df_clusters_file)
matrix_jobs = df_clusters.iloc[:,3:170]

In [3]:
# Chargement des données
df_mean = pd.read_csv(df_mean_file)
matrix_mean = df_mean.iloc[:,1:len(df_mean.columns)]

In [4]:
# Elèves fictifs avec les affinités qu'on pourrait imaginer(technicien, manager, artiste)
# On peut rajouter l'élève obtenu avec le POC "Choix des critères par l'utilisateur"
eleve = ['Compétences sportives', 'Compétences multimédia', 'Planifier', 'Gérer un approvisionnement et des stocks', 'Gérer une production industrielle']

karine1 = [
    'Cultiver ou entretenir des espaces naturels ',
    'Réaliser un art plastique',
    'Jouer de la musique ou chanter',
    'Prendre soin de personnes',
    'Corriger des textes, rédiger, traduire ou collecter des informations',
    'Connaissances du bois',
    'Soins aux personnes',
    'Connaissances artistiques',
    'Connaissances linguistiques',
    'Connaissances littéraires',
    'Compétences artistiques',
    'Compétences d’écriture',
    'Compétences gestuelles',
    'Appréciation sensorielle',
    'Interprétation de données ou de signaux',
    'Compétences mentales',
    'Compétences physiques',
    'Sciences mathématiques',
    'Sciences physico-chimiques',
    'Sciences du vivant',
    'Sciences humaines et sociales',
    'Sciences médicales'
]

marie = ['Animer un groupe',
 'Enseigner',
 'Prendre soin de personnes',
 'Gérer un dossier ou une formalité administrative',
 'Compléter des documents ou des dossiers',
 'Saisir et mettre en forme des documents',
 'Ressources humaines',
 'Compétences artistiques',
 'Interprétation de données ou de signaux',
 'Compétences managériales',
 'Techniques de prévention et gestion de conflits',
 'Techniques pédagogiques',
 'Compétences mentales']

In [5]:
# Transforme les variables de la même manière que sur R
def cast_eleve_R(eleve_to_transform):
    eleve = []
    for feature in eleve_to_transform:
        feature = feature.replace(', ','..')
        feature = feature.replace('(', '.')
        feature = feature.replace(')', '.')
        feature = feature.replace("'", '.')
        feature = feature.replace("/", '.')
        feature = feature.replace(' ', '.')
        eleve.append(feature)
    return(eleve)

### Transforme le résultat du questionnaire en vecteur élève
# Chargement des données de compétences dans la variable comp
comp = pd.read_excel(comp_csv, sheetname="Arbo Comptétences ROMEJuin 2016", header=0)
act = pd.read_excel(act_csv, sheetname='Arbo Activités ROME Juin 2016', header=0)

def cast_eleve(eleve_to_cast):
    eleve_cast = []
    for item in eleve_to_cast:
        item_list = [item]
        if item=='Compétences sportives':
            item_list.append("Techniques de plongée")
        for element in item_list:    
            try:
                index = comp[comp["Libellé questionnaire"]==element]['Niveau 2'].index.tolist()[0]
                if str(comp.iloc[index,1])!='nan':
                    eleve_cast.append(str(comp.iloc[index,1]))
                elif str(comp.iloc[index,2])!='nan':
                    eleve_cast.append(str(comp.iloc[index,2]))
            except:
                pass
            
        for element in item_list:        
            try:
                index = act[act["Libellé questionnaire"]==element]['Niveau 2'].index.tolist()[0]
                if str(act.iloc[index,1])!='nan':
                    eleve_cast.append(str(act.iloc[index,1]))
                elif str(act.iloc[index,2])!='nan':
                    eleve_cast.append(str(act.iloc[index,2]))
            except:
                pass            
    return eleve_cast

#### vecteur élève (petit problème à régler encore)
def vectorize_eleve(eleve_to_vectorize): 
    eleve = pd.DataFrame()
    for var in matrix_mean.columns.tolist():
        if var in eleve_to_vectorize:  
            eleve[var]=[1]
        else:
            eleve[var]=[0]
    return eleve

# Score obtenu par produit scalaire entre cluster et élève

def score_cluster(eleve):
    eleve_c = cast_eleve(eleve)
    eleve_R = cast_eleve_R(eleve_c)
    eleve_v = vectorize_eleve(eleve_R)
    score = matrix_mean.dot(eleve_v.transpose())
    score.columns=["score"]
    score = score.sort_values(by="score",ascending=False)

    jobs_clust = pd.read_csv(jobs_clusters)
    final_scores = pd.DataFrame()
    for row in score.itertuples():
        k = row[0] + 1
        subset = jobs_clust[jobs_clust.clusters==k]
        subset = subset.loc[:,["intitule","clusters"]]
        subset["score"]= [row[1]]*len(subset)
        final_scores = final_scores.append(subset)
    return(final_scores)

def show_score(eleve, n_metiers=20):
    score_eleve = score_cluster(eleve)
    print("--> Préférences de l'élève : \n")
    for item in eleve:
        print(item)
    print("\n --> Métiers recommandés :")
    print(score_eleve[0:n_metiers])

In [7]:
show_score(karine1, 40)

--> Préférences de l'élève : 

Cultiver ou entretenir des espaces naturels 
Réaliser un art plastique
Jouer de la musique ou chanter
Prendre soin de personnes
Corriger des textes, rédiger, traduire ou collecter des informations
Connaissances du bois
Soins aux personnes
Connaissances artistiques
Connaissances linguistiques
Connaissances littéraires
Compétences artistiques
Compétences d’écriture
Compétences gestuelles
Appréciation sensorielle
Interprétation de données ou de signaux
Compétences mentales
Compétences physiques
Sciences mathématiques
Sciences physico-chimiques
Sciences du vivant
Sciences humaines et sociales
Sciences médicales

 --> Métiers recommandés :
                                              intitule  clusters      score
430                     Mannequinat et pose artistique        92  22.551669
432                                              Danse        92  22.551669
433                                   Musique et chant        92  22.551669
434                   

In [13]:
show_score(marie, 20)

--> Préférences de l'élève : 

Animer un groupe
Enseigner
Prendre soin de personnes
Gérer un dossier ou une formalité administrative
Compléter des documents ou des dossiers
Saisir et mettre en forme des documents
Ressources humaines
Compétences artistiques
Interprétation de données ou de signaux
Compétences managériales
Techniques de prévention et gestion de conflits
Techniques pédagogiques
Compétences mentales

 --> Métiers recommandés :
                                              intitule  clusters     score
177      Animation d'activités culturelles ou ludiques        55  9.990682
403                            Enseignement artistique        55  9.990682
404                            Enseignement des écoles        55  9.990682
405               Enseignement général du second degré        55  9.990682
406                             Enseignement supérieur        55  9.990682
407            Enseignement technique et professionnel        55  9.990682
408                 Formation en

In [10]:
show_score(eleve, 20)

--> Préférences de l'élève : 

Compétences sportives
Compétences multimédia
Planifier
Gérer un approvisionnement et des stocks
Gérer une production industrielle

 --> Métiers recommandés :
                                              intitule  clusters     score
121                                       Photographie        42  6.725631
122        Production en laboratoire cinématographique        42  6.725631
123           Production en laboratoire photographique        42  6.725631
447             Montage audiovisuel et post-production        42  6.725631
145                       Direction de chantier du BTP        47  6.137113
317                Intervention en milieu subaquatique        47  6.137113
424                      Management de sécurité privée        47  6.137113
440                              Sportif professionnel        47  6.137113
219  Management et ingénierie gestion industrielle ...        67  5.719522
220  Management et ingénierie méthodes et industria...       